# Quasi-modular Forms mod $p$

Here we make $q$-series calculations of modular forms and their reductions modulo some prime $p$.

## 0. Notations and sets

We fix the following notations and sets:
1. $B[[q]] = \mathbb{Q}[[q]]$ is the power series ring in the variable $q$.
2. $S[P,Q,R] = \mathbb{Q}[x,y,z]$ is the polynomial ring in the variables $x,y,z$.

In [429]:
B.<q> = PowerSeriesRing(QQ,default_prec=300)
S.<P, Q, R> = PolynomialRing(QQ, 3)

## 1. $E_2$, $E_4$, $E_6$ and $\Delta$

Below, we define the Eisenstein series $E_2$, $E_4$ and $E_6$. Then, using these we define $\Delta$. Finally, we define $E_{p-1}$ for a prime $p\geq5$ as a polynomial $A$ in $E_4$ and $E_6$.

In [234]:
# Set precision
prec = 1000

# Define the graded ring of modular forms of level 1
M = ModularFormsRing(1)

# The graded ring is generated as an algebra by the Eisenstein series E4 and E6
E2 = 1-24*sum(sigma(n,1)*q^n for n in range(1,prec+1))
E4 = M.q_expansion_basis(4,prec+1)[0]
E6 = M.q_expansion_basis(6,prec+1)[0]

# Using E4 and E6, we define the Delta function
Delta = (E4^3 - E6^2)/1728;

# Print E4, E6 and Delta up to q^r where
r = 5
print('E_2 =',E2.add_bigoh(r))
print('E_4 =',E4.add_bigoh(r))
print('E_6 =',E6.add_bigoh(r))
print('Delta =',Delta.add_bigoh(r))

E_2 = 1 - 24*q - 72*q^2 - 96*q^3 - 168*q^4 + O(q^5)
E_4 = 1 + 240*q + 2160*q^2 + 6720*q^3 + 17520*q^4 + O(q^5)
E_6 = 1 - 504*q - 16632*q^2 - 122976*q^3 - 532728*q^4 + O(q^5)
Delta = q - 24*q^2 + 252*q^3 - 1472*q^4 + O(q^5)


We define a general Eisenstein series $E_k$.

The function `Ek` has:
- input: positive integers `k`, the weight of $E_k$, and `r`, the precision of the $q$-expansion
- output: the $q$ expansion of $E_k$ up to $q^{r-1}$

In [235]:
def Ek(k,r):
    q_expansion = 1-(2*k/bernoulli(k))*sum(sigma(n,k-1)*q^n for n in range(1,r+2))
    return q_expansion.add_bigoh(r)

In [236]:
Ek(14,6)

1 - 24*q - 196632*q^2 - 38263776*q^3 - 1610809368*q^4 - 29296875024*q^5 + O(q^6)

Next, for a prime $p\geq 5$, we determine the polynomial $A_p\in\mathbb{Q}[x,y]$ for which $A_p(E_4,E_6)=E_{p-1}$.

The function `Ap` has:
- input: a prime $p$
- output: a polynomial in $\mathbb{Q}[x,y]$

## 2. Basis of $M^{\mathrm{qm}}_{\leq k}(\mathbb{Q})$

Given a weight $k$, we compute a monomial basis of $M^{\mathrm{qm}}_{\leq k}(\mathbb{Q})$ which is a set of monomials of the form $E_2^a E_4^b E_6^c$ where $2a+4b+6c\leq k$.

The function `monomial_exponent` has:

- input: a weight `k`
- output: a list of triples `[a,b,c]` with $2a+4b+6c \leq k$

In [237]:
def monomial_exponents(k):
    mon = []
    for a in range(0,k):
        if 2*a<=k:
            for b in range(0,k):
                if 4*b <= k - 2*a:
                    for c in range(0,k):
                        if 6*c <= k-2*a-4*b:
                            mon.append([a,b,c])
                        else: pass
                else: pass
        else: pass
#    mon.pop(0)
    return mon

For example,

In [238]:
k = 6
monomial_exponents(k)

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 0], [2, 0, 0], [3, 0, 0]]

Given a triple $\{a,b,c\}$ of nonnegative integers, we compute $E_2^a E_4^b E_6^b$.

The function `qmform_from_exp` has:
- input: a triple `exponents` of the form `[a,b,c]` where $a,b,c\geq 0$
- output: the $q$-series $E_2^a E_4^b E_6^c$

In [239]:
def qmform_from_exp(exponents):
    return E2^(exponents[0]) * E4^(exponents[1]) * E6^(exponents[2])

For example:

In [240]:
e = [2,3,1]
print('E_2 ^',e[0],'* E_4 ^',e[1],'* E_6 ^',e[2],'has q-series:')
print(qmform_from_exp(e).add_bigoh(6))

E_2 ^ 2 * E_4 ^ 3 * E_6 ^ 1 has q-series:
1 + 168*q - 210168*q^2 - 75792864*q^3 - 7200742584*q^4 + 1060876656*q^5 + O(q^6)


We can write the monomial generating set of $M^{\mathrm{qm}}_{\leq k}(\mathbb{Q})$ as a matrix whose columns are the coefficients of the $q$-expansion of each element of the basis.

The `basis_matrix` has:

- input: a weight `k`
- output: a matrix $(a_{ij})$, where $a_{ij}$ is the $i$th $q$-expansion coefficient of the $j$th basis element.

In [241]:
def basis_matrix(k):
    mons = monomial_exponents(k)
    basis_length = len(mons)
    basis_elements = [qmform_from_exp(u).add_bigoh(basis_length) for u in mons]
    return Matrix([[m.padded_list()[i] for i in range(0,basis_length)] for m in basis_elements]).transpose()

For example

In [242]:
# Fix a weight
k = 6

print(basis_matrix(k))

[       1        1        1        1        1        1        1]
[       0     -504      240      -24      216      -48      -72]
[       0   -16632     2160      -72    -3672      432     1512]
[       0  -122976     6720      -96   -62496     3264    -3744]
[       0  -532728    17520     -168  -322488     9456   -95544]
[       0 -1575504    30240     -144 -1121904    21600  -473904]
[       0 -4058208    60480     -288 -2969568    39744 -1538784]


Here we check that the `basis_matrix(k)` is invertible for even $k$ up to 20.

In [243]:
import time
r = 10 # number of even weights
# For r = 5, runtime ~ 0.05 sec
# For r = 10, runtime ~ 0.9 sec
# For r = 15, runtime ~ 10 sec
# For r = 20, runtime ~ 137 sec


# Record the start time
start_time = time.time()

for k in [2*i for i in range(1, r+1)]:
    det_for_k = basis_matrix(k).det()
    print('the monomial generating set of weights <=',k,'is a basis:',det_for_k!=0)

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('----------------------------------------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

the monomial generating set of weights <= 2 is a basis: True
the monomial generating set of weights <= 4 is a basis: True
the monomial generating set of weights <= 6 is a basis: True
the monomial generating set of weights <= 8 is a basis: True
the monomial generating set of weights <= 10 is a basis: True
the monomial generating set of weights <= 12 is a basis: True
the monomial generating set of weights <= 14 is a basis: True
the monomial generating set of weights <= 16 is a basis: True
the monomial generating set of weights <= 18 is a basis: True
the monomial generating set of weights <= 20 is a basis: True
----------------------------------------------------------------------
Elapsed time: 0.828148 seconds


Checking the determinants modulo a large prime is much quicker though not completely reliable. Below, we check the determinant of `basis_matrix(k)` modulo $p$ where $p$ is the $(k/2)$th prime (this choice of prime was made empirically by computing the factorizations of the determinants and observing that, roughly, new prime factor appeared in the factorization everytime the weight increased by two).

In [244]:
# The prime considered is the kth prime, where k is the weight

r = 5
# For r = 5, runtime ~ 0.06 sec
# For r = 10, runtime ~ 0.9 sec
# For r = 15, runtime ~ 7.5 sec
# For r = 20, runtime ~ 45 sec
# For r = 25, runtime ~ 201 sec

# Record the start time
start_time = time.time()

for k in [2*i for i in range(1, r+1)]:
    p = nth_prime(2*k)
    A = basis_matrix(k).change_ring(GF(p))
    det_modp = det(A)
    print('for weight',k,', the monomial generating set is a basis mod p =',p,'-->',det_modp !=0)

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('----------------------------------------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

for weight 2 , the monomial generating set is a basis mod p = 7 --> True
for weight 4 , the monomial generating set is a basis mod p = 19 --> True
for weight 6 , the monomial generating set is a basis mod p = 37 --> True
for weight 8 , the monomial generating set is a basis mod p = 53 --> True
for weight 10 , the monomial generating set is a basis mod p = 71 --> True
----------------------------------------------------------------------
Elapsed time: 0.052872 seconds


We can write the generating set of $M^{\mathrm{qm}}_{\leq k}(\mathbb{Q})$ as a matrix whose columns are the coefficients of the $q$-expansion of each element of the monomial generating set.

The `basis_matrix` has:

- input: a weight `k` and a positive integer `rows`
- output: a `rows`$\times N$ matrix $(a_{ij})$, where $N$ is the number of monomial generators and $a_{ij}$ is the $i$th $q$-expansion coefficient of the $j$th monomial.

In [245]:
def generating_matrix(k,rows):
    mons = monomial_exponents(k)
    basis_length = len(mons)
    basis_elements = [qmform_from_exp(u).add_bigoh(rows) for u in mons]
    return Matrix([[m.padded_list()[i] for i in range(0,rows)] for m in basis_elements]).transpose()

In [246]:
k = 6
rows = 10

print(generating_matrix(k,rows))

[        1         1         1         1         1         1         1]
[        0      -504       240       -24       216       -48       -72]
[        0    -16632      2160       -72     -3672       432      1512]
[        0   -122976      6720       -96    -62496      3264     -3744]
[        0   -532728     17520      -168   -322488      9456    -95544]
[        0  -1575504     30240      -144  -1121904     21600   -473904]
[        0  -4058208     60480      -288  -2969568     39744  -1538784]
[        0  -8471232     82560      -192  -6737472     66432  -3947328]
[        0 -17047800    140400      -360 -13678200    105840  -8597880]
[        0 -29883672    181680      -312 -24978312    147984 -16987176]


Given an even weight $2k$, we have $$\#\{(a,b,c):2a+4b+6c\leq2k\}=\left\lceil \frac{2k^3+21k^2+66k+30}{72} \right\rceil$$ The function `number_of_monomials` gives this formula.

In [247]:
def number_of_monomials(w):
    if w%2 ==0:
        k = w/2
    else:
        k = (w-1)/2
    return ceil((2*k^3+21*k^2+66*k+30)/72)

In [248]:
k = 96
print('For weight =',k,',')
print('using monomial_exponents gives us',len(monomial_exponents(k)),'many monomials,')
print('using the above formula gives us',number_of_monomials(k),'many monomials.')

For weight = 96 ,
using monomial_exponents gives us 3789 many monomials,
using the above formula gives us 3789 many monomials.


Notice that the function `number_of_monomials` is much faster than computing `len(monomial_exponents(k))`

In [249]:
w = 400

repetitions = 1

import timeit
from math import ceil

# Define a lambda function to include the calculation
calculation_1 = lambda: number_of_monomials(w)
calculation_2 = lambda: len(monomial_exponents(w))

execution_time_1 = timeit.timeit(calculation_1, number=repetitions)
execution_time_2 = timeit.timeit(calculation_2, number=repetitions)

print('weight =',w)
print("Execution time using number_of_monomials:", execution_time_1, "seconds")
print("Execution time using len(monomial_exponents):", execution_time_2, "seconds")

weight = 400
Execution time using number_of_monomials: 2.248800592496991e-05 seconds
Execution time using len(monomial_exponents): 3.162827619991731 seconds


## 3. Generating quasi-modular forms from $\{E_2^a E_4^b E_6^c:a,b,c\geq0\}$

Given a quasi-modular form $f\in M^{\mathrm{qm}(\mathbb{Q})}$ as a $q$-series, we express it as a $\mathbb{Q}$-linear combination of the monomial generating set computed in $\S 2$.

The function `linear_combination_with_prec` has:

- input: a positive even weight `k`, a positive integer `pr` and a $q$-series `f` that is a sum of quasi-modular forms of weight bounded by $k$,

- output: a list of numbers $\{a_1,\ldots,a_n\}$ such that $f=a_1 v_1+\cdots+a_n v_n$ where $\{v_1,\ldots,v_n\}$ is the ordered generating set computed with `monomial_basis`.

In [250]:
def linear_combination_with_prec(k,pr,f):
    mons = monomial_exponents(k)
    basis_length = len(mons)
    if f.common_prec(f) < pr:
        F = f.lift_to_precision(pr)
    else:
        F = f.add_bigoh(pr)
    list_of_coefficients = F.padded_list(pr)
    v = vector([c for c in list_of_coefficients[:pr]])
    A = generating_matrix(k,pr)
    try:
        sol = A.solve_right(v)
    except ValueError:
        return "Error"
    return sol 

In [251]:
k=12
mons = monomial_exponents(k)
print(len(mons))

pr = 29

f = Ek(k,pr);
w = linear_combination_with_prec(12,pr,f)

print(w)

#pol = sum(w[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)

#print(pol)

#print(f-pol(E2,E4,E6))

23
(0, 0, 250/691, 0, 0, 0, 441/691, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


Given a quasi-modular form $f\in M^{\mathrm{qm}(\mathbb{Q})}$ as a $q$-series, we express it as a $\mathbb{Q}$-linear combination of the monomial basis computed in $\S 2$.

The function `linear_comb` has:

- input: a positive even weight `k` and a $q$-series `f` that is a sum of quasi-modular forms of weight bounded by $k$,

- output: a list of numbers $\{a_1,\ldots,a_n\}$ such that $f=a_1 v_1+\cdots+a_n v_n$ where $\{v_1,\ldots,v_n\}$ is the ordered basis computed with `monomial_basis`.

In [252]:
def linear_comb(k,f):
    basis_length = number_of_monomials(k)
    F = f.add_bigoh(basis_length)
    list_of_coefficients = F.padded_list(basis_length)
    v = vector([c for c in list_of_coefficients[:basis_length]])
    A = basis_matrix(k)
    return list(A^(-1)*v)

Given a linear combination of monomials from `linear_comb`, we can check whether the coefficients are $p$-integral.

The function `p_integral_linear_comb` has:
- input: a positive even weight `k`, $q$-series `f` that is a sum of quasi-modular forms of weight bounded by $k$, and a prime `p`

- output: a list of numbers $\{d_1,\ldots,d_n\}$ where $d_1,\ldots,d_n$ are the reduction modulo $p$ of the denominators of the coefficients of the linear combination given by `linear_comb(k,f)`. 

In [253]:
def p_integral_linear_comb(k,f,p):
    return [denominator(u)%p for u in linear_comb(k,f)]

Given a positive even weight $k$, we use `linear_comb` to produce the polynomial in $E_2,E_4$ and $E_6$ that equals $f$.

The function `associated_poly` has:
- input: a positive even weight `k` and a $q$-series `f` that is a sum of quasi-modular forms of weight bounded by $k$,
- output: a polynomial in $P$, $Q$ and $R$ that equals `f` when evaluated at $P=E_2$, $Q=E_4$ and $R=E_6$.

In [254]:
def associated_poly(k,f):
    mons = monomial_exponents(k)
    v = linear_comb(k,f)
    return sum(v[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)

In [255]:
def associated_poly_with_prec(k,pr,f):
    mons = monomial_exponents(k)
    try:
        v = linear_combination_with_prec(k,pr,f)
    except ValueError:
        return "Error"
    else:
        return sum(v[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)

For example:

In [256]:
p = 19 # fixed prime number
k = p-1 # weight of the form
r = 100 # max order when printing q-expansions

f = Ek(k,prec);
pol = associated_poly(k,f)

print('Let:')
print('f = ',f.add_bigoh(10))
print('---------------------------')
print('f is equal to the polynomial:')
print(pol)
print('---------------------------')
print('The difference f-polynomial is:')
print(f-pol(E2,E4,E6))

Let:
f =  1 - 28728/43867*q - 3765465144/43867*q^2 - 3709938631392/43867*q^3 - 493547047383096/43867*q^4 - 21917724609403728/43867*q^5 - 486272786232443616/43867*q^6 - 6683009405824511424/43867*q^7 - 64690198594597187640/43867*q^8 - 479102079577959825624/43867*q^9 + O(q^10)
---------------------------
f is equal to the polynomial:
38367/43867*Q^3*R + 5500/43867*R^3
---------------------------
The difference f-polynomial is:
O(q^1000)


We combine the above functions into the single function `fancy_linear_comb` that has:
- inputs: `k` a highest weight, `f` a $q$-series, `p` a prime, `r` a precision for $q$-series
- ouput: a verbose analysis of `f`

In [257]:
def fancy_linear_comb(k,f,p,r):
    mons = monomial_exponents(k)
    basis_length = len(mons)
    #basis_elements = [qmform_from_exp(u).add_bigoh(basis_length) for u in mons]
    v = linear_comb(k,f)
    associated_poly = sum(v[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)
    denominators_mod_p = p_integral_linear_comb(k,f,p)
    
    if 0 in denominators_mod_p:
        p_integral_check = False
    else:
        p_integral_check = True

    print('Consider the form:')
    print('f =',f.add_bigoh(r))
    print('-----------------------------------------------')
    print('The monomial basis for weights <=',k,"is:")
    print([P^e[0]*Q^e[1]*R^e[2] for e in mons])
    print('-----------------------------------------------')
    print('f is equal to the following linear combination of the above monomial basis:')
    print(associated_poly)
    print('-----------------------------------------------')
    print('The coefficients of the linear combination are:')
    print(v)
    print('-----------------------------------------------')
    print('The denominators modulo',p,'of the denominators of these coefficients are:')
    print(denominators_mod_p)
    print('-----------------------------------------------')
    print('The linear combination is',p,'- integral:',p_integral_check)

For example: we compute the Eisenstein series $E_{p-1}$ as a linear combination of monomials of the form $E_2^a E_4^b E_6^c$.

In [258]:
p = 13 # fixed prime number
k = p-1 # weight of the form
r = 5 # max order when printing q-expansions

basis_length = len(monomial_exponents(k))

f = Ek(k,basis_length); #Define the form

fancy_linear_comb(k,f,p,r)

Consider the form:
f = 1 + 65520/691*q + 134250480/691*q^2 + 11606736960/691*q^3 + 274945048560/691*q^4 + O(q^5)
-----------------------------------------------
The monomial basis for weights <= 12 is:
[1, R, R^2, Q, Q*R, Q^2, Q^3, P, P*R, P*Q, P*Q*R, P*Q^2, P^2, P^2*R, P^2*Q, P^2*Q^2, P^3, P^3*R, P^3*Q, P^4, P^4*Q, P^5, P^6]
-----------------------------------------------
f is equal to the following linear combination of the above monomial basis:
441/691*Q^3 + 250/691*R^2
-----------------------------------------------
The coefficients of the linear combination are:
[0, 0, 250/691, 0, 0, 0, 441/691, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------------------
The denominators modulo 13 of the denominators of these coefficients are:
[1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
-----------------------------------------------
The linear combination is 13 - integral: True


## $\S$4. Finding Graded Components

Given a prime $p$ and a set $X$ of triples $(a,b,c)$, we partition this set into

$$X = \bigsqcup_{i=0}^{p-2} X_i\quad\text{with}\quad X_i :=\{(a,b,c)\in X \mid 2a+4b+6c\equiv i \pmod{p-1}\}$$

The function `graded_exponents_mod_p` takes
- input: A set `X` of triples and a prime number `p`
- output: A set $X_0,X_1,\ldots,X_{p-2}$ of subsets of `X` where $(a,b,c)\in X_i \Longleftrightarrow 2a+4b+6c\equiv i\pmod{p-1}$

In [259]:
def graded_exponents_mod_p(X,p):
    parts = []
    for u in [l for l in range(0,p-2) if l%2 == 0]:
        parts_u = [w for w in X if (2*w[0]+4*w[1]+6*w[2]-u)%(p-1) == 0]
        parts.append(parts_u)
    return parts

In [260]:
p = 13
k = 12 # bound for weight

mons = monomial_exponents(k)

print('-----------------------------------------------------------------------')
print('The set of monomials of weight <=',k,'is')
print([P^e[0]*Q^e[1]*R^e[2] for e in mons])

print('-----------------------------------------------------------------------')
    
g_exp = graded_exponents_mod_p(mons,p)
print('The set of monomials partitioned according to their weights modulo p-1:')

for ge in g_exp:
    print('k =',2*g_exp.index(ge),'-->',[P^t[0]*Q^t[1]*R^t[2] for t in ge])
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
The set of monomials of weight <= 12 is
[1, R, R^2, Q, Q*R, Q^2, Q^3, P, P*R, P*Q, P*Q*R, P*Q^2, P^2, P^2*R, P^2*Q, P^2*Q^2, P^3, P^3*R, P^3*Q, P^4, P^4*Q, P^5, P^6]
-----------------------------------------------------------------------
The set of monomials partitioned according to their weights modulo p-1:
k = 0 --> [1, R^2, Q^3, P*Q*R, P^2*Q^2, P^3*R, P^4*Q, P^6]
k = 2 --> [P]
k = 4 --> [Q, P^2]
k = 6 --> [R, P*Q, P^3]
k = 8 --> [Q^2, P*R, P^2*Q, P^4]
k = 10 --> [Q*R, P*Q^2, P^2*R, P^3*Q, P^5]
-----------------------------------------------------------------------


In [319]:
p = 13
k = 12 # bound for weight

mons = monomial_exponents(k)

print('-----------------------------------------------------------------------')
print('The set of monomials of weight <=',k,'is')
print([P^e[0]*Q^e[1]*R^e[2] for e in mons])

print('-----------------------------------------------------------------------')
    
g_exp = graded_exponents_mod_p(mons,p)

print('The set of monomials partitioned according to their weights modulo p-1:')

for ge in g_exp:
    print('k =',2*g_exp.index(ge),'-->',[P^t[0]*Q^t[1]*R^t[2] for t in ge])
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
The set of monomials of weight <= 12 is
[1, R, R^2, Q, Q*R, Q^2, Q^3, P, P*R, P*Q, P*Q*R, P*Q^2, P^2, P^2*R, P^2*Q, P^2*Q^2, P^3, P^3*R, P^3*Q, P^4, P^4*Q, P^5, P^6]
-----------------------------------------------------------------------
[[[0, 0, 0], [0, 0, 2], [0, 3, 0], [1, 1, 1], [2, 2, 0], [3, 0, 1], [4, 1, 0], [6, 0, 0]], [[1, 0, 0]], [[0, 1, 0], [2, 0, 0]], [[0, 0, 1], [1, 1, 0], [3, 0, 0]], [[0, 2, 0], [1, 0, 1], [2, 1, 0], [4, 0, 0]], [[0, 1, 1], [1, 2, 0], [2, 0, 1], [3, 1, 0], [5, 0, 0]]]
The set of monomials partitioned according to their weights modulo p-1:
k = 0 --> [1, R^2, Q^3, P*Q*R, P^2*Q^2, P^3*R, P^4*Q, P^6]
k = 2 --> [P]
k = 4 --> [Q, P^2]
k = 6 --> [R, P*Q, P^3]
k = 8 --> [Q^2, P*R, P^2*Q, P^4]
k = 10 --> [Q*R, P*Q^2, P^2*R, P^3*Q, P^5]
-----------------------------------------------------------------------


## $\S$5. The Quasi-modular Forms $U_a$

In this section we define the $q$-series $U^{(\beta)}_a$ where $a>1$ and $\beta$ is an ordered list of $(a-1)$ many 0's and 1's. 

First, we convert an ordered list of 0's and 1's into a string of inequalities.

The function `vector_to_ineqs` has:
- input: a list `vec` of 0's and 1's.
- output: a list of strings with "$\geq$" in the place of the 0's and "$>$" in the place of the 1's.

For example, the vector `[0,0,1,1,0]` is mapped to `['>=','>=','>','>','>=']`

In [261]:
inequality_signs = ['>=','>']

def vector_to_ineqs(vec):
    list_of_ineqs = []
    for v in vec:
        if v==0:
            list_of_ineqs.append('>=')
        elif v==1:
            list_of_ineqs.append('>')
        else:
            pass
    return list_of_ineqs

For example,

In [262]:
vec = [0,0,0,1,1,0,1,0,1,1]
print('beta =',vec)
print('associated list of inequalities =',vector_to_ineqs(vec))

beta = [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
associated list of inequalities = ['>=', '>=', '>=', '>', '>', '>=', '>', '>=', '>', '>']


Given a partition $[\lambda_1,\ldots,\lambda_a]$ of $n$ of length $a$, we check whether this partition satisfies the list of inequalities given by a vector $\beta$ with $(a-1)$ entries of 0's and 1's. More precisely:

The function `check_partition_vs_ineqs` has:
- input: a `partition` of $n$ of length $a$, and a list `choice_of_ineqs` of $a-1$ 0's and 1's
- output: `True` or `False` depending on whether the partition $\{\lambda_1,\ldots,\lambda_a\}$ satisfies the given list of inequalities.

In [263]:
def check_partition_vs_ineqs(partition,choice_of_ineqs):
    import itertools
    partition_str = [str(p) for p in partition]
    list_of_zipped_strings = [x for x in itertools.chain.from_iterable(itertools.zip_longest(partition_str,list(choice_of_ineqs))) if x]
    joined_string = "".join(list_of_zipped_strings)
    return eval(joined_string)

For example:

In [264]:
n = 11
a = 4

import random
vec = [random.choice([0, 1]) for _ in range(a-1)]
choice_of_ineqs = vector_to_ineqs(vec)

parts = Partitions(n,length=a)

print('list of inequalities:',choice_of_ineqs)

for p in parts:
    print('The partition',p,'satisfies the inequalites',choice_of_ineqs,':',check_partition_vs_ineqs(p,choice_of_ineqs))

list of inequalities: ['>', '>=', '>']
The partition [8, 1, 1, 1] satisfies the inequalites ['>', '>=', '>'] : False
The partition [7, 2, 1, 1] satisfies the inequalites ['>', '>=', '>'] : False
The partition [6, 3, 1, 1] satisfies the inequalites ['>', '>=', '>'] : False
The partition [6, 2, 2, 1] satisfies the inequalites ['>', '>=', '>'] : True
The partition [5, 4, 1, 1] satisfies the inequalites ['>', '>=', '>'] : False
The partition [5, 3, 2, 1] satisfies the inequalites ['>', '>=', '>'] : True
The partition [5, 2, 2, 2] satisfies the inequalites ['>', '>=', '>'] : False
The partition [4, 4, 2, 1] satisfies the inequalites ['>', '>=', '>'] : False
The partition [4, 3, 3, 1] satisfies the inequalites ['>', '>=', '>'] : True
The partition [4, 3, 2, 2] satisfies the inequalites ['>', '>=', '>'] : False
The partition [3, 3, 3, 2] satisfies the inequalites ['>', '>=', '>'] : False


Given a number $n$, a length $a$ and a list $\beta$ of 0's and 1's, we compute all the partitions of $n$ of size $a$ that satisfy the inequalities associated to $\beta$.

The function `partitions_from_ineqs` has:
- input: a number `n`, a length `a` and a list `vec` of $a-1$ 0's and 1's
- output: the complete list of partitions of $n$ of length $a$ that satisfy the inequalities associated to `vec`.

In [265]:
def partitions_from_ineqs(n,a,vec):
    parts = Partitions(n,length=a)
    choice_of_ineqs = vector_to_ineqs(vec)
    return [p for p in parts if check_partition_vs_ineqs(p,choice_of_ineqs)]

For example:

In [266]:
n = 11
a = 4

import random
vec = [random.choice([0, 1]) for _ in range(a-1)]
choice_of_ineqs = vector_to_ineqs(vec)

print('list of inequalities:',choice_of_ineqs)
print('partitions of',n,'of length',a,'that satisfy',choice_of_ineqs,':')
for p in partitions_from_ineqs(n,a,vec):
    print(p)
    
print(list(partitions_from_ineqs(n,a,vec)[0]))

list of inequalities: ['>', '>=', '>']
partitions of 11 of length 4 that satisfy ['>', '>=', '>'] :
[6, 2, 2, 1]
[5, 3, 2, 1]
[4, 3, 3, 1]
[6, 2, 2, 1]


The $q$-series $U^{(\beta)}_a$ is defined as
$$U^{(\beta)}_a (q) = \sum_{k} \frac{q^{|k|}}{(1-q^k)^2}$$
where we are using multindex notation, i.e.
$$k=(k_1,\ldots,k_a),\quad |k|=k_1+\cdots+k_a,\quad (1-q^k)^2=(1-q^{k_1})^2\cdots(1-q^{k_a})^2$$

The function `U` has
- input: `a` an positive integer, `vec` a list of $a-1$ 0's and 1's and `r` a positive integer
- output: the $q$-series $U^{\beta}_a (q)$ up to $O(q^r)$.

In [267]:
B.<q> = PowerSeriesRing(QQ,default_prec=200)

def U_term(part_index):
    return q^(sum(i for i in part_index))*(prod((1-q^u)^(-2) for u in part_index))

def U(a,vec,r):
    choice_of_ineqs = vector_to_ineqs(vec)
    index_set = []
    for n in range(0,r+1):
        for p in partitions_from_ineqs(n,a,vec):
            index_set.append(list(p))
    return sum(U_term(k) for k in index_set).add_bigoh(r)

For example:

In [306]:
a = 4
vec = [0,1,0]
r = 8

choice_of_ineqs = vector_to_ineqs(vec)
print('list of inequalities:',choice_of_ineqs)

index_set = []
for n in range(0,r+1):
    print('-----------------------------------------------------------------------------')
    print('Partitions of',n,'of length',a,'satisfying',choice_of_ineqs,'is:')
    for p in partitions_from_ineqs(n,a,vec):
        print(p,'-->',U_term(p).add_bigoh(r+2))
        index_set.append(list(p))
print('-----------------------------------------------------------------------------')       
print('The q-series U is:',U(a,vec,r))

list of inequalities: ['>=', '>', '>=']
-----------------------------------------------------------------------------
Partitions of 0 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 1 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 2 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 3 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 4 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 5 of length 4 satisfying ['>=', '>', '>='] is:
-----------------------------------------------------------------------------
Partitions of 6 of length 4 satisfying ['>=', '>

In [308]:
a = 4
vec = [0,0,0]
r = 20
f = U(a,vec,r)
pol = associated_poly(2*a,f)

print('Size of monomial basis:',len(monomial_exponents(2*a)))
print('---------------------------')
print('Let:')
print('f = ',f)
print('---------------------------')
print('The associated polynomial is:')
print('F =',pol)
print('---------------------------')
print('The difference f-F is:')
print(f-pol(E2,E4,E6))

Size of monomial basis: 11
---------------------------
Let:
f =  q^4 + 9*q^5 + 44*q^6 + 156*q^7 + 450*q^8 + 1121*q^9 + 2508*q^10 + 5139*q^11 + 9867*q^12 + 17831*q^13 + 30888*q^14 + 51117*q^15 + 82212*q^16 + 127458*q^17 + 193809*q^18 + 285702*q^19 + O(q^20)
---------------------------
The associated polynomial is:
F = 1/7962624*P^4 + 5/1990656*P^3 + 1/3317760*P^2*Q + 37/2211840*P^2 + 1/331776*P*Q + 19/116121600*Q^2 + 1/4354560*P*R + 5/172032*P + 37/5529600*Q + 1/870912*R - 27859/464486400
---------------------------
The difference f-F is:
O(q^20)


To test the quasi-modularity of $U^{(\beta)}_a$ for all binary lists $\beta$, we have the following function that lists all possible such $\beta$.

In [272]:
def generate_binary_lists(length):
    all_binary_lists = []
    total_possibilities = 2**length
    for i in range(total_possibilities):
        binary_str = bin(i)[2:].zfill(length)  # Convert to binary and pad with zeros
        binary_list = [int(bit) for bit in binary_str]
        all_binary_lists.append(binary_list)
    
    return all_binary_lists

For example,

In [273]:
a = 4
generate_binary_lists(a-1)

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

Given $a$, there are $2^{a-1}$ different $U^{(\beta)}_a$. For each of these, we compute the associated polynomial and the difference between the evaluated associated polynomial and $U^{(\beta)}_a$.

In [274]:
a = 4
all_vectors = generate_binary_lists(a-1)
mon_size = len(monomial_exponents(2*a))
r = mon_size +10

print('Size of monomial generating set:',mon_size)

for vec in all_vectors:
    f = U(a,vec,r)
    pol = associated_poly(2*a,f)
    print('-----------------------------')
    print('For U _',a,'^',vec,'=',f.add_bigoh(f.valuation()+1),', then U - associated_poly(U) =',(f-pol(E2,E4,E6)).add_bigoh(mon_size+1))

Size of monomial generating set: 11
-----------------------------
For U _ 4 ^ [0, 0, 0] = q^4 + O(q^5) , then U - associated_poly(U) = O(q^12)
-----------------------------
For U _ 4 ^ [0, 0, 1] = q^7 + O(q^8) , then U - associated_poly(U) = -3/20*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [0, 1, 0] = q^6 + O(q^7) , then U - associated_poly(U) = -61/20*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [0, 1, 1] = q^9 + O(q^10) , then U - associated_poly(U) = -9/4*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [1, 0, 0] = q^5 + O(q^6) , then U - associated_poly(U) = 16/5*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [1, 0, 1] = q^8 + O(q^9) , then U - associated_poly(U) = 61/20*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [1, 1, 0] = q^7 + O(q^8) , then U - associated_poly(U) = -4/5*q^11 + O(q^12)
-----------------------------
For U _ 4 ^ [1, 1, 1] = q^10 + O(q^11) , then U - associated_poly(U) = O(q^12)


In [275]:
def check_all_U_for_fixed_a(a):
    all_vectors = generate_binary_lists(a-1)
    mon_size = len(monomial_exponents(2*a))
    r = mon_size + 1
    quasi_modular_forms = []
    for vec in all_vectors:
        f = U(a,vec,r)
        pol = associated_poly(2*a,f)
        if (f-pol(E2,E4,E6)).truncate(mon_size+1)==0:
            quasi_modular_forms.append(vec)
        else:
            pass
    return quasi_modular_forms

def check_all_U_for_fixed_a_verbose(a):
    all_vectors = generate_binary_lists(a-1)
    mon_size = len(monomial_exponents(2*a))
    r = mon_size + 1
    print('Size of monomial generating set:',mon_size)
    print('--------------------------------------')
    for vec in all_vectors:
        f = U(a,vec,r)
        pol = associated_poly(2*a,f)
        print('For U _',a,'^',vec,'=',f.add_bigoh(f.valuation()+1),', then U - associated_poly(U) =',(f-pol(E2,E4,E6)).add_bigoh(mon_size+1))
        print('--------------------------------------')
    return ''

In [276]:
a = 4

print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')
for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

print('------------------------------------------------')
print('Long Answer:')
print('--------------------------------------')
print(check_all_U_for_fixed_a_verbose(a))

Short Answer:
--------------------------------------
Inequalities for which U _ 4 is quasi-modular:
['>=', '>=', '>=']
['>', '>', '>']
------------------------------------------------
Long Answer:
--------------------------------------
Size of monomial generating set: 11
--------------------------------------
For U _ 4 ^ [0, 0, 0] = q^4 + O(q^5) , then U - associated_poly(U) = O(q^12)
--------------------------------------
For U _ 4 ^ [0, 0, 1] = q^7 + O(q^8) , then U - associated_poly(U) = -3/20*q^11 + O(q^12)
--------------------------------------
For U _ 4 ^ [0, 1, 0] = q^6 + O(q^7) , then U - associated_poly(U) = -61/20*q^11 + O(q^12)
--------------------------------------
For U _ 4 ^ [0, 1, 1] = q^9 + O(q^10) , then U - associated_poly(U) = -9/4*q^11 + O(q^12)
--------------------------------------
For U _ 4 ^ [1, 0, 0] = q^5 + O(q^6) , then U - associated_poly(U) = 16/5*q^11 + O(q^12)
--------------------------------------
For U _ 4 ^ [1, 0, 1] = q^8 + O(q^9) , then U - associate

In [277]:
a = 5

print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')
for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

print('------------------------------------------------')
print('Long Answer:')
print('--------------------------------------')
print(check_all_U_for_fixed_a_verbose(a))

Short Answer:
--------------------------------------
Inequalities for which U _ 5 is quasi-modular:
['>=', '>=', '>=', '>=']
['>', '>', '>', '>']
------------------------------------------------
Long Answer:
--------------------------------------
Size of monomial generating set: 16
--------------------------------------
For U _ 5 ^ [0, 0, 0, 0] = q^5 + O(q^6) , then U - associated_poly(U) = O(q^17)
--------------------------------------
For U _ 5 ^ [0, 0, 0, 1] = q^9 + O(q^10) , then U - associated_poly(U) = -16/5*q^16 + O(q^17)
--------------------------------------
For U _ 5 ^ [0, 0, 1, 0] = q^8 + O(q^9) , then U - associated_poly(U) = 56/11*q^16 + O(q^17)
--------------------------------------
For U _ 5 ^ [0, 0, 1, 1] = q^12 + O(q^13) , then U - associated_poly(U) = 103/11*q^16 + O(q^17)
--------------------------------------
For U _ 5 ^ [0, 1, 0, 0] = q^7 + O(q^8) , then U - associated_poly(U) = -1646/55*q^16 + O(q^17)
--------------------------------------
For U _ 5 ^ [0, 1, 0, 1]

In [278]:
#a = 6

#print('Short Answer:')
#print('--------------------------------------')
#print('Inequalities for which U _',a,'is quasi-modular:')
#for x in check_all_U_for_fixed_a(a):
#    print(vector_to_ineqs(x))
#
#print('------------------------------------------------')
#print('Long Answer:')
#print('--------------------------------------')
#print(check_all_U_for_fixed_a_verbose(a))

In [279]:
# <1 sec for a=2
a = 2
import time


print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
start_time = time.time()

for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('--------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Short Answer:
--------------------------------------
Inequalities for which U _ 2 is quasi-modular:
['>=']
['>']
--------------------------------------
Elapsed time: 0.009499 seconds


In [280]:
# <1 sec for a=3
a = 3
import time


print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
start_time = time.time()

for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('--------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Short Answer:
--------------------------------------
Inequalities for which U _ 3 is quasi-modular:
['>=', '>=']
['>', '>']
--------------------------------------
Elapsed time: 0.093534 seconds


In [281]:
# <1 sec for a=4
a = 4
import time


print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
start_time = time.time()

for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('--------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Short Answer:
--------------------------------------
Inequalities for which U _ 4 is quasi-modular:
['>=', '>=', '>=']
['>', '>', '>']
--------------------------------------
Elapsed time: 0.169416 seconds


In [282]:
# <1 sec for a=5
a = 5
import time


print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
start_time = time.time()

for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('--------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Short Answer:
--------------------------------------
Inequalities for which U _ 5 is quasi-modular:
['>=', '>=', '>=', '>=']
['>', '>', '>', '>']
--------------------------------------
Elapsed time: 0.828292 seconds


In [283]:
# ~4.5 sec for a=6
a = 6
import time


print('Short Answer:')
print('--------------------------------------')
print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
start_time = time.time()

for x in check_all_U_for_fixed_a(a):
    print(vector_to_ineqs(x))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('--------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Short Answer:
--------------------------------------
Inequalities for which U _ 6 is quasi-modular:
['>=', '>=', '>=', '>=', '>=']
['>', '>', '>', '>', '>']
--------------------------------------
Elapsed time: 4.501560 seconds


In [284]:
# ~29 sec for a=7
#a = 7
#import time

#print('Short Answer:')
#print('--------------------------------------')
#print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
#start_time = time.time()

#for x in check_all_U_for_fixed_a(a):
#    print(vector_to_ineqs(x))

# Record the end time
#end_time = time.time()

# Calculate and print the elapsed time
#elapsed_time = end_time - start_time
#print('--------------------------------------')
#print(f"Elapsed time: {elapsed_time:.6f} seconds")

In [285]:
# ~260 sec for a=8
#a = 8
#import time

#print('Short Answer:')
#print('--------------------------------------')
#print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
#start_time = time.time()

#for x in check_all_U_for_fixed_a(a):
#    print(vector_to_ineqs(x))

# Record the end time
#end_time = time.time()

# Calculate and print the elapsed time
#elapsed_time = end_time - start_time
#print('--------------------------------------')
#print(f"Elapsed time: {elapsed_time:.6f} seconds")

In [286]:
# ~2972 sec ~ 50min for a=9
#a = 9
#import time


#print('Short Answer:')
#print('--------------------------------------')
#print('Inequalities for which U _',a,'is quasi-modular:')

# Record the start time
#start_time = time.time()

#for x in check_all_U_for_fixed_a(a):
#    print(vector_to_ineqs(x))

# Record the end time
#end_time = time.time()

# Calculate and print the elapsed time
#elapsed_time = end_time - start_time
#print('--------------------------------------')
#print(f"Elapsed time: {elapsed_time:.6f} seconds")

---

## Scratch

In [287]:
print('1 =',1)
print('E2 =',E2.add_bigoh(5))
print('E4 =',E4.add_bigoh(5))
print('E2^2 =',(E2*E2).add_bigoh(5))
print('E6 =',E6.add_bigoh(5))
print('E2*E4 =',(E2)*(E4).add_bigoh(5))
print('E2^3 =',(E2^3).add_bigoh(5))

1 = 1
E2 = 1 - 24*q - 72*q^2 - 96*q^3 - 168*q^4 + O(q^5)
E4 = 1 + 240*q + 2160*q^2 + 6720*q^3 + 17520*q^4 + O(q^5)
E2^2 = 1 - 48*q + 432*q^2 + 3264*q^3 + 9456*q^4 + O(q^5)
E6 = 1 - 504*q - 16632*q^2 - 122976*q^3 - 532728*q^4 + O(q^5)
E2*E4 = 1 + 216*q - 3672*q^2 - 62496*q^3 - 322488*q^4 + O(q^5)
E2^3 = 1 - 72*q + 1512*q^2 - 3744*q^3 - 95544*q^4 + O(q^5)


In [305]:
k = 10; print('Diagonal Basis for Quasimodular forms of mixed weight <=',k)
d = number_of_monomials(k); print('Dimension of the space =',d)

mons_exp = monomial_exponents(k)
mons = [qmform_from_exp(u) for u in mons_exp]

A = basis_matrix(k)
all_primes_in_all_denominators = []

def flatten(x):
    return [item for sublist in x if sublist for item in sublist]

print('----------------------------------------------------')

for r in range(0,d):
    v = vector([0]*d)
    v[r] = 1
    sol = A.solve_right(v)
    q_series = sum(sol[i]*mons[i] for i in range(0,d))
    dens = set([denominator(a) for a in q_series.padded_list()])
    primes_in_denominator = []
    
    for w in dens:
        primes_in_denominator.append(prime_factors(w))
    
    flattened_primes_in_denominators = flatten(primes_in_denominator)
    if flattened_primes_in_denominators == []:
        all_primes_in_dens = []
    else:
        all_primes_in_dens = list(set(flattened_primes_in_denominators));
        all_primes_in_all_denominators.append(all_primes_in_dens)
    print(q_series.add_bigoh(d+1))
    print('denominators =',dens)
    print('prime factors of the denominators =',all_primes_in_dens)
    print('----------------------------------------------------')
    
print('all prime factors in all denominators:',list(set(flatten(all_primes_in_all_denominators))))

Diagonal Basis for Quasimodular forms of mixed weight <= 10
Dimension of the space = 16
----------------------------------------------------
1 + O(q^17)
denominators = {1}
prime factors of the denominators = []
----------------------------------------------------
q - 4239/5*q^16 + O(q^17)
denominators = {1, 34, 2, 5, 170, 10, 17, 85}
prime factors of the denominators = [17, 2, 5]
----------------------------------------------------
q^2 + 3624/11*q^16 + O(q^17)
denominators = {1, 11, 187, 17}
prime factors of the denominators = [17, 11]
----------------------------------------------------
q^3 + 3723/5*q^16 + O(q^17)
denominators = {1, 85, 5, 17}
prime factors of the denominators = [17, 5]
----------------------------------------------------
q^4 - 3486/5*q^16 + O(q^17)
denominators = {1, 85, 5, 17}
prime factors of the denominators = [17, 5]
----------------------------------------------------
q^5 + 147*q^16 + O(q^17)
denominators = {1, 2}
prime factors of the denominators = [2]
--------

In [289]:
prime_factors(12)

[2, 3]

In [290]:
factor(2055673503011323086129126293797)

7^2 * 181 * 2274779977 * 101891999837845369

In [291]:
def theta(g):
    return q * derivative(g,q,1)

def iter_theta(g,n):
    for _ in range(1, n+1):
        g = theta(g)
    return g

In [292]:
k = 8; print('weight <=',k)
r = number_of_monomials(k); print('upper bound on the dimension =',r)
p = 13; print('prime =',p)

print('------------------------------------------------------------')

f = E2^3 + E2*E6 + E2*E4 + E2; print('f = ')
print(f.add_bigoh(r+1))

print('------------------------------------------------------------')

fbar = f.change_ring(GF(p)); print('f modulo',p,'=')
print(fbar.add_bigoh(r+1))

print('------------------------------------------------------------')

for n in range(1,3):
    nth_der_of_f = iter_theta(f,n)
    nth_der_of_f_modp = nth_der_of_f.change_ring(GF(11))
    print('derivative #',n,'of fbar is:')
    print(nth_der_of_f_modp.add_bigoh(r+1))
    print('------------------------------------------------------------')

weight <= 8
upper bound on the dimension = 11
prime = 13
------------------------------------------------------------
f = 
4 - 408*q - 6840*q^2 + 246048*q^3 + 3246216*q^4 + 20149488*q^5 + 82273824*q^6 + 266018880*q^7 + 719517960*q^8 + 1717003464*q^9 + 3723294384*q^10 + 7445445984*q^11 + O(q^12)
------------------------------------------------------------
f modulo 13 =
4 + 8*q + 11*q^2 + 10*q^3 + 12*q^4 + 8*q^5 + 9*q^6 + 10*q^7 + 5*q^8 + 7*q^9 + 4*q^10 + 2*q^11 + O(q^12)
------------------------------------------------------------
derivative # 1 of fbar is:
10*q + 4*q^2 + 2*q^4 + 2*q^5 + 3*q^6 + 9*q^7 + q^8 + 4*q^10 + O(q^12)
------------------------------------------------------------
derivative # 2 of fbar is:
10*q + 8*q^2 + 8*q^4 + 10*q^5 + 7*q^6 + 8*q^7 + 8*q^8 + 7*q^10 + O(q^12)
------------------------------------------------------------


In [293]:
k = 8; print('weight <=',k)
r = number_of_monomials(k); print('upper bound on the dimension =',r)
p = 3; print('prime =',p)

print('------------------------------------------------------------')

f = E2^3 + E2*E6 + E2*E4 + E2; print('f = ')
print(f.add_bigoh(r+1))

print('------------------------------------------------------------')

print('U_p(f) =')
print(Up(f,p).add_bigoh(r+1))

print('------------------------------------------------------------')

print('V_p(f) =')
print(f.V(p).add_bigoh(r+1))
print('------------------------------------------------------------')

print('T_5(f) =')
print(Tn(8,f,5).add_bigoh(r+1))

weight <= 8
upper bound on the dimension = 11
prime = 3
------------------------------------------------------------
f = 
4 - 408*q - 6840*q^2 + 246048*q^3 + 3246216*q^4 + 20149488*q^5 + 82273824*q^6 + 266018880*q^7 + 719517960*q^8 + 1717003464*q^9 + 3723294384*q^10 + 7445445984*q^11 + O(q^12)
------------------------------------------------------------
U_p(f) =
4 + 246048*q + 82273824*q^2 + 1717003464*q^3 + 14026659360*q^4 + 69892592832*q^5 + 259154146728*q^6 + 774529860864*q^7 + 2012944834080*q^8 + 4618148400960*q^9 + 9800687468736*q^10 + 19122885714816*q^11 + O(q^12)
------------------------------------------------------------
V_p(f) =
4 - 408*q^3 - 6840*q^6 + 246048*q^9 + O(q^12)
------------------------------------------------------------
T_5(f) =
312504 + 20149488*q + 3723294384*q^2 + 69892592832*q^3 + 549803275056*q^4 + 2674774290528*q^5 + 9800687468736*q^6 + 28980925640064*q^7 + 74901797947440*q^8 + 170863467945264*q^9 + 361381667824224*q^10 + 702385848207936*q^11 + O(q^12)


In [294]:
def Up(g,p):
    B.<q> = PowerSeriesRing(QQ,default_prec=200)
    coef_list = g.padded_list()
    new_coef_list = coef_list[0::p]
    return B(new_coef_list)

def Tn(k,g,ell):
    return Up(g,ell) + ell^(k-1) * (g.V(ell))

In [295]:
def det_mod_p(A,p):
    A_modp = A.change_ring(GF(p))
    return det(A_modp)

In [296]:
r = 10

# Record the start time
start_time = time.time()

for k in [2*i for i in range(1, r+1)]:
    A = basis_matrix(k)
    i = 1
    while det_mod_p(A,nth_prime(i)) == 0:
        i += 1
    print('for weights <=',k,', the first prime with nonzero determinant is p =',nth_prime(i),', with deteriminant =',det_mod_p(A,nth_prime(i)))

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print('----------------------------------------------------------------------')
print(f"Elapsed time: {elapsed_time:.6f} seconds")

for weights <= 2 , the first prime with nonzero determinant is p = 5 , with deteriminant = 1
for weights <= 4 , the first prime with nonzero determinant is p = 7 , with deteriminant = 4
for weights <= 6 , the first prime with nonzero determinant is p = 11 , with deteriminant = 4
for weights <= 8 , the first prime with nonzero determinant is p = 11 , with deteriminant = 8
for weights <= 10 , the first prime with nonzero determinant is p = 13 , with deteriminant = 8
for weights <= 12 , the first prime with nonzero determinant is p = 17 , with deteriminant = 3
for weights <= 14 , the first prime with nonzero determinant is p = 17 , with deteriminant = 4
for weights <= 16 , the first prime with nonzero determinant is p = 19 , with deteriminant = 5
for weights <= 18 , the first prime with nonzero determinant is p = 23 , with deteriminant = 8
for weights <= 20 , the first prime with nonzero determinant is p = 23 , with deteriminant = 13
-------------------------------------------------------

In [297]:
f = (1/3)*q
f.change_ring(GF(5))

2*q

In [298]:
k = 8; print('weight <=',k)
r = number_of_monomials(k); print('upper bound on the dimension =',r)
p = 5; print('prime =',p)

print('------------------------------------------------------------')

f = E2^3 + E2*E6 + E2*E4 + E2; print('f = ')
print(f.add_bigoh(r+1))
print('f = ',associated_poly(k,f))

print('------------------------------------------------------------')

f = fbar = f.change_ring(GF(p)); print('f modulo',p,'=')
print(fbar.add_bigoh(r+1))


print('------------------------------------------------------------')


list_of_coefficients = fbar.padded_list(r)
v = vector([c for c in list_of_coefficients])
A = basis_matrix(k).change_ring(GF(p))
try:
    sol = A.solve_right(v)
    print(sol)
except ValueError:
    print("Error")
    
print('------------------------------------------------------------')

solQQ = [u.lift() for u in sol]
mons = monomial_exponents(k)
poly = sum(solQQ[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)

print(poly)

weight <= 8
upper bound on the dimension = 11
prime = 5
------------------------------------------------------------
f = 
4 - 408*q - 6840*q^2 + 246048*q^3 + 3246216*q^4 + 20149488*q^5 + 82273824*q^6 + 266018880*q^7 + 719517960*q^8 + 1717003464*q^9 + 3723294384*q^10 + 7445445984*q^11 + O(q^12)
f =  P^3 + P*Q + P*R + P
------------------------------------------------------------
f modulo 5 =
4 + 2*q + 3*q^3 + q^4 + 3*q^5 + 4*q^6 + 4*q^9 + 4*q^10 + 4*q^11 + O(q^12)
------------------------------------------------------------
(0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0)
------------------------------------------------------------
P^3 + P*R + 2*R


In [299]:
def associated_poly(k,f):
    mons = monomial_exponents(k)
    v = linear_comb(k,f)
    return sum(v[mons.index(e)]*P^e[0]*Q^e[1]*R^e[2] for e in mons)

In [351]:
a = 3
vec = [0,0]
r = 20
f = U(a,vec,r)

p = 7; print('Let p =',p) # fixed prime number
k = 2*a # weight of the form

print('-----------------------------------------------------------------------')

fancy_linear_comb(k,f,p,r)

mons = monomial_exponents(k)

print('-----------------------------------------------------------------------')
    
g_exp = graded_exponents_mod_p(mons,p)
print('The set of monomials partitioned according to their weights modulo p-1:')

for ge in g_exp:
    print('k =',2*g_exp.index(ge),'-->',[P^t[0]*Q^t[1]*R^t[2] for t in ge])
print('-----------------------------------------------------------------------')

poly = associated_poly(k,f)

for ge in g_exp:
    graded_poly = sum(poly.coefficient({P:m[0], Q:m[1], R:m[2]})*P^(m[0])*Q^(m[1])*R^(m[2]) for m in ge)
    print('k =',2*g_exp.index(ge),'-->',graded_poly)
print('-----------------------------------------------------------------------')

for ge in g_exp:
    graded_poly = sum(poly.coefficient({P:m[0], Q:m[1], R:m[2]})*P^(m[0])*Q^(m[1])*R^(m[2]) for m in ge)
    print('k =',2*g_exp.index(ge),'-->',graded_poly(E2,E4,E6).add_bigoh(a+1))
print('-----------------------------------------------------------------------')

Let p = 7
-----------------------------------------------------------------------
Consider the form:
f = q^3 + 7*q^4 + 27*q^5 + 77*q^6 + 181*q^7 + 378*q^8 + 707*q^9 + 1254*q^10 + 2052*q^11 + 3290*q^12 + 4928*q^13 + 7371*q^14 + 10381*q^15 + 14756*q^16 + 19818*q^17 + 27158*q^18 + 35139*q^19 + O(q^20)
-----------------------------------------------
The monomial basis for weights <= 6 is:
[1, R, Q, P, P*Q, P^2, P^3]
-----------------------------------------------
f is equal to the following linear combination of the above monomial basis:
-1/82944*P^3 - 1/9216*P^2 - 1/69120*P*Q - 1/5120*P - 1/23040*Q - 1/181440*R + 367/967680
-----------------------------------------------
The coefficients of the linear combination are:
[367/967680, -1/181440, -1/23040, -1/5120, -1/69120, -1/9216, -1/82944]
-----------------------------------------------
The denominators modulo 7 of the denominators of these coefficients are:
[0, 0, 3, 3, 2, 4, 1]
-----------------------------------------------
The linear c

In [370]:
640%7

3

In [367]:
a = 4
vec = [0]*(a-1)
r = 20
f = U(a,vec,r)

p = 7; print('Let p =',p) # fixed prime number
k = 2*a # weight of the form

print('-----------------------------------------------------------------------')

fancy_linear_comb(k,f,p,r)

mons = monomial_exponents(k)

print('-----------------------------------------------------------------------')
    
g_exp = graded_exponents_mod_p(mons,p)
print('The set of monomials partitioned according to their weights modulo p-1:')

for ge in g_exp:
    print('k =',2*g_exp.index(ge),'-->',[P^t[0]*Q^t[1]*R^t[2] for t in ge])
print('-----------------------------------------------------------------------')

poly = associated_poly(k,f)

for i in (0..a):
    f_i = poly.coefficient(P^i)
    print('f_',i,'=',f_i)
print('-----------------------------------------------------------------------')

for i in (0..a):
    fi = poly.coefficient(P^i)(E2,E4,E6)
    print('f_',i,'=',B(fi).add_bigoh(5))
print('-----------------------------------------------------------------------')

Let p = 7
-----------------------------------------------------------------------
Consider the form:
f = q^4 + 9*q^5 + 44*q^6 + 156*q^7 + 450*q^8 + 1121*q^9 + 2508*q^10 + 5139*q^11 + 9867*q^12 + 17831*q^13 + 30888*q^14 + 51117*q^15 + 82212*q^16 + 127458*q^17 + 193809*q^18 + 285702*q^19 + O(q^20)
-----------------------------------------------
The monomial basis for weights <= 8 is:
[1, R, Q, Q^2, P, P*R, P*Q, P^2, P^2*Q, P^3, P^4]
-----------------------------------------------
f is equal to the following linear combination of the above monomial basis:
1/7962624*P^4 + 5/1990656*P^3 + 1/3317760*P^2*Q + 37/2211840*P^2 + 1/331776*P*Q + 19/116121600*Q^2 + 1/4354560*P*R + 5/172032*P + 37/5529600*Q + 1/870912*R - 27859/464486400
-----------------------------------------------
The coefficients of the linear combination are:
[-27859/464486400, 1/870912, 37/5529600, 19/116121600, 5/172032, 1/4354560, 1/331776, 37/2211840, 1/3317760, 5/1990656, 1/7962624]
----------------------------------------

In [414]:
A = basis_matrix(4)
print(A,'\n')

print('det A=',factor(det(A)))

Abar = Matrix([[1/det(A),1,1,1],[0,240,-24,-48],[0,2160,-72,432],[0,6720,-96,3264]])

v = vector([0,0,1,0])

sol = Abar^(-1)*v

print(Abar^(-1)*v)

q_series = sol[0]*1/det(A) + sol[1]*E4 + sol[2]*E2 + sol[3]*E2^2

print(q_series.add_bigoh(20))

#B = Matrix([[1,1/240,1/24,1/28],[0,1,-1,-1],[0,9,3,9],[0,28,-4,68]])

#print(Abar^(-1))

#print(B)

#print('det B =',factor(det(B)))

#print(B^(-1))

[   1    1    1    1]
[   0  240  -24  -48]
[   0 2160  -72  432]
[   0 6720  -96 3264] 

det A= 2^15 * 3^5 * 5
(-1050624, 1/480, 1/36, -1/288)
q^2 - q^4 - 16*q^5 - 20*q^6 - 64*q^7 - 85*q^8 - 144*q^9 - 210*q^10 - 320*q^11 - 364*q^12 - 560*q^13 - 712*q^14 - 832*q^15 - 1085*q^16 - 1344*q^17 - 1595*q^18 - 1920*q^19 + O(q^20)


In [472]:
2^15 * 3^5 * 5

39813120

In [409]:
4064256-7/1440-5/48+1/144

1950842831/480

In [415]:
for k in (1..4):
    A = basis_matrix(2*k)
    d = det(A)
    print('for weights <=',2*k,'the determinant is:',factor(abs(d)))

for weights <= 2 the determinant is: 2^3 * 3
for weights <= 4 the determinant is: 2^15 * 3^5 * 5
for weights <= 6 the determinant is: 2^40 * 3^16 * 5^3 * 7
for weights <= 8 the determinant is: 2^88 * 3^34 * 5^8 * 7^3


In [419]:
def ff(x,*y):
    return print(x,y[0],y[1],y[2])

In [438]:
def qseries_matrix(r,li):
    return Matrix([g.padded_list(r) for g in li]).transpose()

In [496]:
k = 4
i = 1 # canonical basis vector index
r = number_of_monomials(k)

print('weights up to:',k,', the space has dimension',r)
print('------------------------------------------------------')

B.<q> = PowerSeriesRing(QQ,default_prec=300)
d = 1/det(basis_matrix(k))

mons = monomial_exponents(k) # exponents of monomial basis
mons.pop(0) # remove the constant monomial
monomial_basis = [B(d)]+[qmform_from_exp(m) for m in mons]

A = qseries_matrix(len(monomial_basis),monomial_basis)

zeros = [0]*r
zeros[i-1] = 1
v = vector(zeros) # ith canonical basis vector

sol = A^(-1)*v

print('linear combination of modified monomial basis required to generate q ^',i-1,'+ ... is:\n')
print(sol)
print('\n-----------------------------------------------------')

diag_q_series = sum(sol[i]*monomial_basis[i] for i in (0..r-1))

print('resulting q-series:\n')
print(diag_q_series.add_bigoh(r+1))

weights up to: 4 , the space has dimension 4
------------------------------------------------------
linear combination of modified monomial basis required to generate q ^ 0 + ... is:

(39813120, 0, 0, 0)

-----------------------------------------------------
resulting q-series:

1 + O(q^5)


In [511]:
B.<q> = PowerSeriesRing(QQ,default_prec=300)

for l in (1..7):
    k = 2*l
    r = number_of_monomials(k)

    print('------------------------------------------------------------------')
    print('Diagonal basis for QM forms of weights <=',k,'of dimension',r,':\n')
    d = 1/det(basis_matrix(k))

    mons = monomial_exponents(k) # exponents of monomial basis
    mons.pop(0) # remove the constant monomial
    monomial_basis = [B(d)]+[qmform_from_exp(m) for m in mons]

    A = qseries_matrix(len(monomial_basis),monomial_basis)

    for i in (1..r):
        zeros = [0]*r
        zeros[i-1] = 1
        v = vector(zeros) # ith canonical basis vector

        sol = A^(-1)*v

        diag_q_series = sum(sol[i]*monomial_basis[i] for i in (0..r-1))

        print(diag_q_series.add_bigoh(r+2))

------------------------------------------------------------------
Diagonal basis for QM forms of weights <= 2 of dimension 2 :

1 + O(q^4)
q + 3*q^2 + 4*q^3 + O(q^4)
------------------------------------------------------------------
Diagonal basis for QM forms of weights <= 4 of dimension 4 :

1 + O(q^6)
q - 2*q^4 + 18*q^5 + O(q^6)
q^2 - q^4 - 16*q^5 + O(q^6)
q^3 + 3*q^4 + 9*q^5 + O(q^6)
------------------------------------------------------------------
Diagonal basis for QM forms of weights <= 6 of dimension 7 :

1 + O(q^9)
q - 41*q^7 - 63*q^8 + O(q^9)
q^2 + 16*q^7 + 10*q^8 + O(q^9)
q^3 + 45/2*q^7 + 135/2*q^8 + O(q^9)
q^4 - 20*q^7 - 45*q^8 + O(q^9)
q^5 + 5/2*q^7 - 5/2*q^8 + O(q^9)
q^6 + 3*q^7 + 9*q^8 + O(q^9)
------------------------------------------------------------------
Diagonal basis for QM forms of weights <= 8 of dimension 11 :

1 + O(q^13)
q - 27/20*q^11 - 2114/5*q^12 + O(q^13)
q^2 + 46/5*q^11 + 1078/5*q^12 + O(q^13)
q^3 + 9/2*q^11 + 271*q^12 + O(q^13)
q^4 - 18*q^11 - 322*q^